In [3]:
%load_ext autoreload
%autoreload 2

# Systematically Improving RAG - Week 4

## Applying Our Topics

In the previous notebook, we used topic modelling to generate topics from our dataset. Through that we realised that there were a significant number of queries related to refunds that we were performing poorly on. We also learnt that there were broadly four different types of queries when it came to refunds

1. Queries about the refund status of their order
2. Queries about whether their order was eligible for a refund
3. Queries about wanting a refund instead of store credit
4. Queries to ask for help because the store didn't respond to their requests

We want to take these insights and build out a classifier that can assign these four types of queries to new queries so that we can monitor and improve our performance on them over time. We can do so using a simple `.yaml` file that contains the categories we've identified over time.

We've defined a few categories in our `categories.yml` file that we'll use to classify our queries. This makes it easy to add new categories in the future and allow other members of the team to add new categories as they see fit.

## Using pre-defined categories to classify queries

We'll start by loading in our categories from the `categories.yml` file. We'll use instructor and pydantic here to handle the classification.


In [24]:
import yaml
from pydantic import BaseModel, ValidationInfo, field_validator


class QuestionType(BaseModel):
    types: list[str]

    @field_validator("types")
    def validate_categories(cls, v: list[str], info: ValidationInfo):
        context = info.context
        if not context:
            raise ValueError("No context provided")
        question_types = context.get("question_types")
        if not question_types:
            raise ValueError("No question types provided")

        question_type_names = [question_type["title"] for question_type in question_types]

        for question_type in v:
            if question_type not in question_type_names:
                raise ValueError(
                    f"Question type {question_type} not found in original list of question types"
                )
        return v


yaml_config = yaml.load(open("categories.yml", "r"), Loader=yaml.FullLoader)

question_types = []
for question_type_category in yaml_config["question_type"]:
    question_types.extend(yaml_config["question_type"][question_type_category])

question_types[:4]


[{'title': 'Return/Refund Process',
  'description': 'User is asking how to initiate a return or refund for their order'},
 {'title': 'Return/Refund Status',
  'description': 'User is inquiring about the status of their return or refund'},
 {'title': 'Eligibility',
  'description': 'User wants to know if their order is eligible for a return or refund'},
 {'title': 'Policy Details',
  'description': "User is asking about the company's return and refund policies"}]

We want to then load in these categories into our prompt. `instructor` allows us to use the same variable in our template formatting and our validation, making it easy to validate that our generated categories are valid.

In [23]:
import openai
import instructor
from rich import print


client = instructor.from_openai(openai.OpenAI())


def classify_query(query: str) -> QuestionType:
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """
            Breakdown the folowing query into the relevant question types. Select all that apply to the query itself.
            
            Question Types:
            {% for question_type in question_types %}
            - {{ question_type.title }} : {{ question_type.description }}
            {% endfor %}
            
            Make sure to only return the categories that are in the list of categories.
            """,
            },
            {"role": "user", "content": query},
        ],
        response_model=QuestionType,
        context={"question_types": question_types},
    )


queries = [
    "What's the status of my refund?",
    "Nike isn't responding to me at all despite the broken shoebag, does Klarna provide any warranty support?",
    "Will Klarna cover the cost of returning my order? I'm out of state and I need to ship it to New York",
    "How many litres of milk does a cow produce in a day?",
]

for query in queries:
    print(classify_query(query))


QuestionType(types=['Return/Refund Status'])

QuestionType(types=['Store Response', 'Warranty Information', 'Others'])

QuestionType(types=['Eligibility', 'Policy Details', 'Shipping Information'])

QuestionType(types=['Others'])

We want to run this in production so that we can monitor and check our performance on the different question types over time.

This is especially true in this case for the refund categories where we're performing poorly. We can make this easier by building dashboards where we can 

- Track the distribution of these question types over time
- Track the % of others to detect drift in our system
- Track satisfaction scores and volume for each query type

and so on.

By investing in this analysis on a consistent basis, we can make sure that we're able to improve our system time by identifying and prioritising the queries that we need to improve on.